# Explore here

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv")
pd

<module 'pandas' from '/Users/nandapop/Library/Python/3.8/lib/python/site-packages/pandas/__init__.py'>